In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

image_path = 'C:/Users/tabm9/OneDrive - Caissa Analytica/Documents/DS_Portfolio/Projects/Images/RLCS'

# Test with only 2 teams

In [ ]:
def run_games(Pa=1/4, K=1, N=[10000, 1], thres=0.10, stop_burnout=False):
    # Run N games
    Sa = (np.random.random(N) < Pa).astype(int)

    # Calculate Ratings
    Ra = np.zeros(N)
    pa = np.zeros(N)
    pa[1] += 1/2
    finished_burnout = np.array([False] * N[1])
    n_to_convergence = np.ones(N[1]) * (N[0] + 1)
    for i in range(1, N[0]):
        # Update rating
        Ra[i] = Ra[i-1] + K * (Sa[i - 1] - pa[i - 1])

        # Update Approx Probabilities
        pa[i] = 1 / (1 + 10 ** (-2 * Ra[i] / 400))

        # Check if burnout is finished
        if not all(finished_burnout):
            condition = abs(pa[i] - Pa) <= Pa * thres
            finished_burnout = np.where(condition, True, finished_burnout)
            n_to_convergence = np.where(finished_burnout, n_to_convergence, i + 1)

            if all(finished_burnout) and stop_burnout:
                break

    return pa.T, n_to_convergence
        

In [ ]:
def logbase(base, x):
    return np.log(x) / np.log(base)

def K_generator(K_interval=[0,400], K_default=300):
    K_d = (K_default - K_interval[0]) / (K_interval[1] - K_interval[0])
    b = logbase(1/2, 1/2 - np.log((1 + np.e ** (-2 * np.e) - K_d) / (K_d + np.e ** (-2 * np.e))) / (4 * np.e))

    def K(x, b=b, K_interval=K_interval):
        return (K_interval[1] - K_interval[0]) * ((1 + 2 * np.e ** (-2 * np.e)) / (1 + np.e ** (-4 * np.e * (x ** b - 1/2))) - np.e ** (-2 * np.e)) + K_interval[0]

    return K



def run_games_improved(Pa=1/4, K_interval=[0,200], K_default=180, N=[10000, 1], thres=0.10, thres_n=10, stop_burnout=False):
    # Run N games
    Sa = (np.random.random(N) < Pa).astype(int)

    # Calculate Ratings
    Ra = np.zeros(N)
    pa = np.zeros(N)
    pa[1] += 1/2
    fpa_list = pa.copy()
    finished_burnout = np.array([False] * N[1])
    n_to_convergence = np.ones(N[1]) * (N[0] + 1)
    K = K_generator(K_interval=K_interval, K_default=K_default)
    K_list = -np.ones(N)

    for i in range(1, N[0]):
        # Update rating
        frequentist_pa = Sa[:i].sum(0) / i
        
        K_i = K(abs(pa[i - 1] - frequentist_pa))
        Ra[i] = Ra[i-1] + K_i * (Sa[i - 1] - pa[i - 1])

        K_list[i] = K_i
        fpa_list[i] = frequentist_pa

        # Update Approx Probabilities
        pa[i] = 1 / (1 + 10 ** (-2 * Ra[i] / 400))
        # print(K_i, Ra[i], pa[i])

        # Check if burnout is finished
        if not all(finished_burnout) and i >= thres_n:
            condition = (abs(pa[i - thres_n: i] - Pa) <= Pa * thres).T.all(1)
            finished_burnout = np.where(condition, True, finished_burnout)
            n_to_convergence = np.where(finished_burnout, n_to_convergence, i + 1 - thres_n)

            if all(finished_burnout) and stop_burnout:
                break

    return pa.T, n_to_convergence, fpa_list.T

In [ ]:
Pa = 1/4
N = [3000, 1000]
N_range = list(range(N[0] + 1))
# pa, n_to_convergence = run_games(Pa=Pa, N=N)
pa_improved, n_to_convergence_improved, fpa_list = run_games_improved(Pa=Pa, N=N)

# print(n_to_convergence_improved)

fig = make_subplots(rows = 2)

fig.add_trace(go.Scatter(x=N_range, y=[Pa] * (N[0] + 1), line=dict(color='black'), name='Real Probability'), row=1, col=1)

for i in range(N[1]):
    if i == 0:
        # fig.add_trace(go.Scatter(x=N_range, y=pa[i], opacity=np.sqrt(1 / N[1]), line=dict(color='blue'), name='Approximated Probability'), row=1, col=1)
        fig.add_trace(go.Scatter(x=N_range, y=pa_improved[i], opacity=np.sqrt(1 / N[1]), line=dict(color='red'), name='Approximated Probability with inertia'), row=1, col=1)
    else:
        # fig.add_trace(go.Scatter(x=N_range, y=pa[i], opacity=np.sqrt(1 / N[1]), line=dict(color='blue'), showlegend=False), row=1, col=1)
        fig.add_trace(go.Scatter(x=N_range, y=pa_improved[i], opacity=np.sqrt(1 / N[1]), line=dict(color='red'), showlegend=False), row=1, col=1)

# fig.add_trace(go.Scatter(x=list(range(N[0])), y=fpa_list[0], opacity=0.5, name='Frequentist Probability'), row=1, col=1)

fig.add_trace(go.Histogram(x=n_to_convergence_improved, nbinsx=100), row=2, col=1)

fig.show()